# X-ray Absorption Spectroscopy (XAS) Experiment at SCS

Author: Jun Zhu jun.zhu@xfel.eu

Created on Nov. 12, 2018

Copyright (C) European X-Ray Free-Electron Laser Facility GmbH.

All rights reserved.

## Introduction

<table><tr>
    <td>
        <img src='misc/mono.png' width="600 px">
        <figcaption>Higley et al., Rev. Sci. Instrum.  87, 033110 (2016)</figcaption>
    </td>
    <td>
        <img src='misc/illustration.png' width="920 px">
    </td>
    <td>
        <img src='misc/expected_data.png' width="800 px">
        <figcaption>Higley et al., submitted</figcaption></td>
</tr></table>

## Data Analysis

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from karabo_data import RunDirectory

from xas_processor import XasFastADC

In [ ]:
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.figsize'] = (8.0, 6.0)

In [ ]:
!ls /gpfs/exfel/exp/SCS/201831/p900048/raw

In [ ]:
run_folder = "/gpfs/exfel/exp/SCS/201831/p900048/raw/r0150"
run = RunDirectory(run_folder)
run.info()

In [ ]:
xgm_device = 'SCS_BLU_XGM/XGM/DOOCS'

adc_device = 'SCS_UTC1_MCP/ADC/1'
adc_channels = {name: "{}:channel_{}.output".format(adc_device, ch) 
                for name, ch in zip(["MCP1", "MCP2", "MCP3", "MCP4"], [9, 8, 7, 6])}

xas = XasFastADC(xgm_id=xgm_device, adc_id=adc_device, adc_channels=adc_channels)

## Sanity check

*check whether the pulse energy in the pipeline data is calibrated*

In [ ]:
idx = -1
tid, data = run.train_from_index(idx)

In [ ]:
xgm_run = run.get_dataframe(fields=[(xgm_device, '*value')])
xgm_run.rename(columns=lambda x: x.split('/')[-1], inplace=True)

# pulse_energy_train_resolved = xgm_run['pulseEnergy.photonFlux'].iloc[idx]
# pulse_energy_pulse_resolved = data[xgm_device + ':output']['data.intensityTD'][
#     0:int(xgm_run['pulseEnergy.nummberOfBrunches'].iloc[idx])].mean()

# print("Train resolved pulse energy is: {:.4f} micro J".format(pulse_energy_train_resolved))
# print("Pulse resolved pulse energy is: {:.4f} micro J".format(pulse_energy_pulse_resolved))
# if abs(pulse_energy_train_resolved - pulse_energy_pulse_resolved) > 0.01*pulse_energy_train_resolved:
#     print("Warning: pulse resolved pulse energy is not calibrated!")

***check the numbers of pulses and train resolved pulse energies throughtout the run***

In [ ]:
_, ax = plt.subplots(figsize=(12, 4.5))
ax2 = ax.twinx()

ln1 = ax.plot(xgm_run['pulseEnergy.nummberOfBrunches'], label="number of pulses", c='g')
ln2 = ax2.plot(xgm_run['pulseEnergy.photonFlux'], label=r"pulse energy ($\mu$J)")

lns = ln1 + ln2
lables = [l.get_label() for l in lns]
ax.legend(lns, lables, loc=0)
ax.set_xlabel("Train ID")
ax.set_ylabel("Number of pulses")
ax2.set_ylabel(r"Pulse energy ($\mu$J)")

plt.tight_layout()

***check that the interested FastAdc channels all have data***

In [ ]:
fig, axes = plt.subplots(len(adc_channels), 1, figsize=(14, 5*len(adc_channels)))

for ax, channel in zip(axes, adc_channels):
    ax.plot(data[list(adc_channels.values())[2]]['data.rawData'] - 
            data[list(adc_channels.values())[2]]['data.baseline'], '.-')
    ax.set_title(channel)
    ax.set_ylabel("intensity (arb.)")

ax.set_xlabel("samples")
ax.set_xlim(1500, 1700)

print("Trigger time: {}".format(data[adc_device]['triggerTime.value']))
print("Delay: {}".format(data[adc_device]['delay.value']))

## Correlation between I0 and I1

In [ ]:
xas.process_run(run_folder)

xas.plot_correlation()

## Absorption spectrum